In [4]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('data/news/dataset.csv')


#keep only the columns we need - title and date
df = df[['Title', 'Date']]

# Display the first few rows of the DataFrame
df.head()

,Title,Date
0,"ATMs to become virtual bank branches, accept d...","May 26, 2020, Tuesday"
1,IDFC First Bank seniors to forgo 65% of bonus ...,"May 26, 2020, Tuesday"
2,"Huge scam in YES Bank for many years, says Enf...","May 25, 2020, Monday"
3,"Bank of Maharashtra sanctioned Rs 2,789 cr in ...","May 24, 2020, Sunday"
4,DCB Bank's profit before tax declines 37.6% to...,"May 23, 2020, Saturday"


In [5]:
# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Filter the DataFrame for dates between 2018 and 2020
filtered_df = df[(df['Date'] >= '2018-01-01') & (df['Date'] <= '2020-12-31')]

# Display the first few rows of the filtered DataFrame
filtered_df.head()

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv('filtered_dataset.csv', index=False)

# Display the shape of the filtered DataFrame
print(filtered_df.shape)

(4726, 2)


In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

sentences = ["there is a shortage of capital, and we need extra financing",  
             "growth is strong and we have plenty of liquidity", 
             "there are doubts about our finances", 
             "profits are flat"]
results = nlp(sentences)
print(results)  #LABEL_0: neutral; LABEL_1: positive; LABEL_2: negative


C:\Users\shubham\AppData\Roaming\Python\Python311\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


[{'label': 'Negative', 'score': 0.9966173768043518}, {'label': 'Positive', 'score': 1.0}, {'label': 'Negative', 'score': 0.9999710321426392}, {'label': 'Neutral', 'score': 0.9889442920684814}]


In [6]:
# Analyze sentiment for each news title in the filtered DataFrame
# filtered_df['Sentiment'] = filtered_df['Title'].apply(lambda x: nlp(x)[0]['label'])
# filtered_df['Sentiment_Score'] = filtered_df['Title'].apply(lambda x: nlp(x)[0]['score'])

# Display the first few rows of the updated DataFrame
filtered_df.head()

C:\Users\shubham\AppData\Local\Temp\ipykernel_16176\3659432128.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Sentiment'] = filtered_df['Title'].apply(lambda x: nlp(x)[0]['label'])
C:\Users\shubham\AppData\Local\Temp\ipykernel_16176\3659432128.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Sentiment_Score'] = filtered_df['Title'].apply(lambda x: nlp(x)[0]['score'])


,Title,Date,Sentiment,Sentiment_Score
0,"ATMs to become virtual bank branches, accept d...",2020-05-26,Neutral,0.999943
1,IDFC First Bank seniors to forgo 65% of bonus ...,2020-05-26,Neutral,0.951466
2,"Huge scam in YES Bank for many years, says Enf...",2020-05-25,Neutral,0.996678
3,"Bank of Maharashtra sanctioned Rs 2,789 cr in ...",2020-05-24,Neutral,0.995092
4,DCB Bank's profit before tax declines 37.6% to...,2020-05-23,Negative,0.999972


In [7]:
filtered_df.to_csv('filtered_dataset_with_sentiment.csv', index=False)

In [ ]:
import openai

# Set up your OpenAI API key
openai.api_key = 'your-api-key-here'

# Define a function to get a response from OpenAI API
def get_openai_response(prompt):
    """
    Generates a response from OpenAI's GPT-3 model based on the provided prompt.

    Args:
        prompt (str): The input text prompt to generate a response for.

    Returns:
        str: The generated response text from the model.
    """
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=100
    )
    return response.choices[0].text.strip()

# Loop over the top 10 headlines in the DataFrame
for headline in df['Title'].head(10):
    prompt = f"""
    Extract the following features from the news headline:
    1. Named Entities
    2. Topic
    3. Sector/Industry
    4. Financial Metrics
    5. Tone

    Headline: "{headline}"
    """
    response = get_openai_response(prompt)
    print(f"Headline: {headline}\nResponse: {response}\n")

| **News** | **Sentiment** | **Named Entities** | **Topic** | **Sector/Industry** | **Financial Metrics** | **Tone** |
|-------------------------------------------|----------------|-----------------------------|-------------------------|------------------------|--------------------------|------------------|
| NBFC yields yet to show contraction...    | Neutral        | NBFC, Govt                  | Spreads                  | Finance                 | None                     | Neutral          |
| Sitharaman gives liquidity boost...       | Positive       | Sitharaman, Shadow Banks, MFIs | Liquidity Boost         | Finance                 | Rs 75,000 cr             | Optimistic       |
| Kotak Bank profit before tax slips...     | Negative       | Kotak Bank                  | Earnings Report          | Banking                 | 10.6% decrease, Rs 2,674 cr | Cautionary    |
| Differentiated bank holdco norms...       | Positive       | Bandhan, IDFC First          | Regulation Change        | Banking                 | None                     | Optimistic       |
| Bad bank may start with Rs 60K-crore...   | Neutral        | Bad Bank, Govt               | NPA Management           | Banking                 | Rs 60K-crore NPAs         | Neutral          |
| Some MSMEs may need deep restructuring... | Negative       | MSMEs, Union Bank            | Restructuring            | Finance                 | None                     | Cautionary       |
| IBA considering proposal to set up...     | Positive       | IBA, PSBs                   | NPA Management           | Banking                 | None                     | Optimistic       |
| Not offering any emergency loan...        | Neutral        | SBI, YONO                   | Clarification            | Banking                 | None                     | Neutral          |


## After filtering merge the news for each day

In [2]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('data/news/sentiment.csv')

# Display the first few rows of the DataFrame
df.head()

,Title,Date,Sentiment,Sentiment_Score
0,"ATMs to become virtual bank branches, accept d...",2020-05-26,Neutral,0.999943
1,IDFC First Bank seniors to forgo 65% of bonus ...,2020-05-26,Neutral,0.951466
2,"Huge scam in YES Bank for many years, says Enf...",2020-05-25,Neutral,0.996678
3,"Bank of Maharashtra sanctioned Rs 2,789 cr in ...",2020-05-24,Neutral,0.995092
4,DCB Bank's profit before tax declines 37.6% to...,2020-05-23,Negative,0.999972


In [4]:
from collections import Counter

# Assuming df is already defined and contains the necessary data
# Group by 'Date' and aggregate the sentiments
grouped_df = df.groupby('Date')['Sentiment'].apply(lambda x: ', '.join(x)).reset_index()

# Function to determine the most common sentiment for a given list of sentiments
def most_common_sentiment(sentiments):
    return Counter(sentiments.split(', ')).most_common(1)[0][0]

# Apply the function to the 'Sentiment' column
grouped_df['Daily_Sentiment'] = grouped_df['Sentiment'].apply(most_common_sentiment)

# Display the first few rows of the updated DataFrame
grouped_df.head(20)

# Drop the sentiment column
grouped_df = grouped_df.drop(columns=['Sentiment'])

# Save the updated DataFrame to a new CSV file
grouped_df.to_csv('daily_sentiment.csv', index=False)

In [5]:
# Find the missing dates in the DataFrame
missing_dates = pd.date_range(start=grouped_df['Date'].min(), end=grouped_df['Date'].max()).difference(grouped_df['Date'])
print(missing_dates)

# Create a DataFrame for the missing dates with the most common sentiment
most_common_sentiment = grouped_df['Daily_Sentiment'].mode()[0]
missing_dates_df = pd.DataFrame({'Date': missing_dates, 'Daily_Sentiment': most_common_sentiment})

# Concatenate the missing dates DataFrame with the grouped DataFrame
updated_grouped_df = pd.concat([grouped_df, missing_dates_df]).sort_values(by='Date').reset_index(drop=True)

# Display the first few rows of the updated DataFrame
updated_grouped_df.head(20)

DatetimeIndex(['2018-01-21', '2018-01-28', '2018-01-29', '2018-02-02',
               '2018-02-11', '2018-04-29', '2018-06-14', '2018-07-14',
               '2018-08-12', '2018-08-19', '2018-10-27', '2018-12-01',
               '2019-01-20', '2019-02-16', '2019-02-23', '2019-02-28',
               '2019-03-19', '2019-03-21', '2019-04-07', '2019-04-15',
               '2019-05-12', '2019-05-18', '2019-06-16', '2019-06-23',
               '2019-06-26', '2019-07-21', '2019-07-26', '2019-07-27',
               '2019-10-13', '2019-11-10', '2019-11-23', '2020-02-02',
               '2020-04-25', '2020-05-16'],
              dtype='datetime64[ns]', freq=None)


TypeError: '<' not supported between instances of 'Timestamp' and 'str'

In [28]:
# Concatenate the missing dates DataFrame with the grouped DataFrame
updated_grouped_df = pd.concat([grouped_df, missing_dates_df]).sort_values(by='Date').reset_index(drop=True)

# Display the first few rows of the updated DataFrame
updated_grouped_df.head(20)

# Save the updated DataFrame to a new CSV file
# updated_grouped_df.to_csv('updated_daily_sentiment.csv', index=False)

TypeError: '<' not supported between instances of 'Timestamp' and 'str'